<a href="https://colab.research.google.com/github/shivamoop/logo_detection/blob/main/logo_detection_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q ultralytics opencv-python pandas numpy yt-dlp



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 86.7 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
import cv2, pandas, numpy

print("Ultralytics version:", YOLO)
print("OpenCV version:", cv2.__version__)
print("Environment ready on Colab")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics version: <class 'ultralytics.models.yolo.model.YOLO'>
OpenCV version: 4.12.0
Environment ready on Colab


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
import os

ADIDAS_PATH = "/content/drive/MyDrive/Adidas-alpha.v1-adidas.yolov8-20251219T130219Z-1-001/Adidas-alpha.v1-adidas.yolov8"
Nike_PATH   = "/content/drive/MyDrive/nike.v1-nike.yolov8"


print("Adidas files:", len(os.listdir(ADIDAS_PATH)))
print("Nike files:", len(os.listdir(Nike_PATH)))

Adidas files: 6
Nike files: 5


In [9]:
for item in os.listdir(Nike_PATH):
    print(item)

train
valid
README.dataset.txt
README.roboflow.txt
data.yaml


In [10]:
for item in os.listdir(ADIDAS_PATH):
    print(item)

train
valid
test
README.roboflow.txt
data.yaml
README.dataset.txt


In [11]:
BASE = "/content/data_merged"
for split in ["train", "valid", "test"]:
    os.makedirs(f"{BASE}/images/{split}", exist_ok=True)
    os.makedirs(f"{BASE}/labels/{split}", exist_ok=True)

print("Unified dataset folders created")


Unified dataset folders created


In [12]:
import shutil

def fast_merge(src_root, class_id, prefix):
    for split in ["train", "valid"]:
        img_src = os.path.join(src_root, split, "images")
        lbl_src = os.path.join(src_root, split, "labels")

        img_dst = os.path.join(BASE, "images", split)
        lbl_dst = os.path.join(BASE, "labels", split)

        for img_name in os.listdir(img_src):
            if not img_name.lower().endswith((".jpg", ".png", ".jpeg")):
                continue

            new_img = f"{prefix}_{img_name}"
            shutil.copy2(
                os.path.join(img_src, img_name),
                os.path.join(img_dst, new_img)
            )

            lbl_name = os.path.splitext(img_name)[0] + ".txt"
            src_lbl = os.path.join(lbl_src, lbl_name)
            dst_lbl = os.path.join(lbl_dst, os.path.splitext(new_img)[0] + ".txt")

            with open(src_lbl, "r") as r, open(dst_lbl, "w") as w:
                for line in r:
                    parts = line.strip().split()
                    w.write(f"{class_id} " + " ".join(parts[1:]) + "\n")

fast_merge(ADIDAS_PATH, 0, "adidas")
fast_merge(Nike_PATH, 1, "Nike")

print("Fast merge completed")


Fast merge completed


In [13]:
with open(f"{BASE}/data.yaml", "w") as f:
    f.write(
f"""path: {BASE}
train: images/train
val: images/valid
test: images/test

names:
  0: Adidas
  1: Nike
"""
)


In [14]:
def count_images(path):
    return len([f for f in os.listdir(path) if f.lower().endswith((".jpg",".png",".jpeg"))])

def count_labels(path):
    return len([f for f in os.listdir(path) if f.lower().endswith(".txt")])

for split in ["train", "valid", "test"]:
    img_count = count_images(f"{BASE}/images/{split}")
    lbl_count = count_labels(f"{BASE}/labels/{split}")
    print(f"{split.upper()} → Images: {img_count}, Labels: {lbl_count}")


TRAIN → Images: 166, Labels: 166
VALID → Images: 48, Labels: 48
TEST → Images: 0, Labels: 0


In [15]:
print("Sample label file:")
sample = os.listdir(f"{BASE}/labels/train")[0]
with open(f"{BASE}/labels/train/{sample}") as f:
    print(f.read())


Sample label file:
0 0.3090717299578059 0.8108298171589311 0.1371308016877637 0.06047819971870605



In [16]:
DATA_YAML = "/content/data_merged/data.yaml"


In [17]:


model = YOLO("yolov8s.pt")
model.info()


YOLOv8s summary: 129 layers, 11,166,560 parameters, 0 gradients, 28.8 GFLOPs


(129, 11166560, 0, 28.816844800000002)

In [20]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
!nvidia-smi


CUDA available: True
CUDA device count: 1
Fri Dec 19 13:13:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----

In [21]:
model = YOLO("yolov8s.pt")

model.train(
    data="/content/data_merged/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    optimizer="AdamW",
    lr0=1e-3,
    weight_decay=5e-4,
    patience=10,
    device=0,
    project="logo_detection",
    name="yolov8s_adidas_puma"
)


Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data_merged/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_adidas_puma, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=10, perspective=0.0, plot

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d67e95df2f0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [22]:
model = YOLO("/content/logo_detection/yolov8s_adidas_puma/weights/best.pt")
metrics = model.val(data="/content/data_merged/data.yaml")
print(metrics)

Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 725.2±499.1 MB/s, size: 60.3 KB)
val: Scanning /content/data_merged/labels/valid.cache... 48 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 48/48 17.2Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 1.3it/s 2.3s
                   all         48         61      0.745      0.552      0.612      0.394
                Adidas         38         44      0.928      0.876      0.968      0.673
                  Nike         10         17      0.562      0.228      0.257      0.115
Speed: 9.6ms preprocess, 13.5ms inference, 0.0ms loss, 4.4ms postprocess per image
Results saved to /content/runs/detect/val
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultra

In [23]:
metrics.box.map, metrics.box.map50, metrics.box.mp, metrics.box.mr


(np.float64(0.394205077552588),
 np.float64(0.6124870998017334),
 np.float64(0.7449969954860415),
 np.float64(0.5516347584901466))

In [24]:
model.predict(
    source="/content/data_merged/images/valid",
    conf=0.25,
    save=True,
    project="logo_detection",
    name="val_preview"
)



image 1/48 /content/data_merged/images/valid/Nike_download-1-_jpg.rf.c93b048ecb85b4e0c8b29bd099412bad.jpg: 448x640 (no detections), 75.9ms
image 2/48 /content/data_merged/images/valid/Nike_download-3-_jpg.rf.b0c98eed9a58c213aeeaafac432a5fe7.jpg: 640x480 (no detections), 67.4ms
image 3/48 /content/data_merged/images/valid/Nike_download-8-_jpg.rf.7eb234dc4acf47ada0d0b3e31410a24a.jpg: 480x640 2 Nikes, 66.2ms
image 4/48 /content/data_merged/images/valid/Nike_images-17-_jpg.rf.465c0802cd073dcfeaf118bc8a7eed13.jpg: 640x416 2 Nikes, 66.4ms
image 5/48 /content/data_merged/images/valid/Nike_images-20-_jpg.rf.3575c2dece94e6c879cf6a58c94fa088.jpg: 640x640 1 Nike, 17.1ms
image 6/48 /content/data_merged/images/valid/Nike_images-21-_jpg.rf.193c07fd3ee9a460cd194c80b6b633b0.jpg: 640x640 (no detections), 17.0ms
image 7/48 /content/data_merged/images/valid/Nike_images-24-_jpg.rf.48ed437b87f4727d34c4b2f9534e400e.jpg: 640x480 1 Nike, 12.9ms
image 8/48 /content/data_merged/images/valid/Nike_images-8-_jpg.

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'Adidas', 1: 'Nike'}
 obb: None
 orig_img: array([[[219, 184, 150],
         [219, 184, 150],
         [219, 184, 150],
         ...,
         [158, 179, 180],
         [159, 180, 178],
         [159, 180, 178]],
 
        [[220, 185, 151],
         [220, 185, 151],
         [220, 185, 151],
         ...,
         [173, 194, 195],
         [174, 195, 193],
         [174, 195, 193]],
 
        [[221, 186, 153],
         [221, 186, 153],
         [221, 186, 153],
         ...,
         [167, 188, 189],
         [168, 189, 187],
         [168, 189, 187]],
 
        ...,
 
        [[149, 172, 180],
         [102, 125, 133],
         [172, 195, 203],
         ...,
         [ 38,  36,  36],
         [ 38,  36,  36],
         [ 38,  36,  36]],
 
        [[ 76,  99, 107],
         [ 86, 109, 117],
         [157, 180, 188],
         ...,
       

In [32]:
import cv2

model = YOLO("/content/logo_detection/yolov8s_adidas_puma/weights/best.pt")

cap = cv2.VideoCapture("/content/drive/MyDrive/input_cvat.mp4")

video_fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(
    "/content/drive/MyDrive/output_annotated.mp4",
    cv2.VideoWriter_fourcc(*"mp4v"),
    video_fps,
    (w, h)
)

frame_id = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # ✅ Process only 1 frame per second
    if frame_id % video_fps == 0:
        results = model(frame, conf=0.25)[0]
        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            label = model.names[cls]

            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
            cv2.putText(frame, f"{label} {conf:.2f}",
                        (x1, y1-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                        (0,255,0), 2)

    out.write(frame)
    frame_id += 1

cap.release()
out.release()



0: 384x640 (no detections), 11.8ms
Speed: 2.6ms preprocess, 11.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 14.3ms
Speed: 3.0ms preprocess, 14.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.2ms
Speed: 4.6ms preprocess, 13.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.5ms
Speed: 4.9ms preprocess, 11.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.8ms
Speed: 5.3ms preprocess, 10.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.6ms
Speed: 3.4ms preprocess, 13.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.9ms
Speed: 4.6ms preprocess, 10.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Adidas, 11.3ms
Speed: 3.8ms preprocess, 11.3ms inferenc

In [33]:
import csv

model = YOLO("/content/logo_detection/yolov8s_adidas_puma/weights/best.pt")

cap = cv2.VideoCapture("/content/drive/MyDrive/input_cvat.mp4")
video_fps = int(cap.get(cv2.CAP_PROP_FPS))

csv_path = "/content/drive/MyDrive/detections.csv"
with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["frame_id", "timestamp_sec", "class", "confidence", "x1", "y1", "x2", "y2"])

    frame_id = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # ✅ 2 FPS sampling
        if frame_id % video_fps  == 0:
            results = model(frame, conf=0.15)[0]  # LOWER conf
            t = frame_id / video_fps

            for box in results.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = model.names[int(box.cls[0])]
                conf = float(box.conf[0])
                writer.writerow([frame_id, round(t, 2), cls, round(conf, 4), x1, y1, x2, y2])

        frame_id += 1

cap.release()

print("CSV saved at:", csv_path)



0: 384x640 (no detections), 11.3ms
Speed: 2.2ms preprocess, 11.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.7ms
Speed: 2.9ms preprocess, 10.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.8ms
Speed: 3.1ms preprocess, 11.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.7ms
Speed: 2.5ms preprocess, 10.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.7ms
Speed: 3.1ms preprocess, 10.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.7ms
Speed: 3.7ms preprocess, 10.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.8ms
Speed: 3.8ms preprocess, 10.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Adidas, 10.8ms
Speed: 5.1ms preprocess, 10.8ms inferenc

In [35]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/detections.csv")
df.head(), len(df)


(   frame_id  timestamp_sec   class  confidence    x1   y1    x2    y2
 0       210            7.0  Adidas      0.2654  1799  960  1870  1030
 1       240            8.0  Adidas      0.3246   906  937   987  1000
 2       240            8.0  Adidas      0.2142  1686  400  1800   486
 3       240            8.0  Adidas      0.1976   972  936  1040   996
 4       300           10.0    Nike      0.5239   877  552   982   617,
 996)